# 前言

tmall_order_report.csv 这个数据集是订单数据，可供挖掘的纬度有订单时间、省份（收货地址），指标则有销售量、销售额、退款金额、退货率、成交率、地区分布、下单时间趋势等。

## 第一步：数据清洗

### 1.加载数据：使用 pandas 读取 CSV 文件。

In [1]:
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker
from pyecharts.datasets import register_url

In [2]:
data = pd.read_csv('tmall_order_report.csv')
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


### 2.处理缺失值：检查并处理缺失值。

In [3]:
data.columns = data.columns.str.strip()  # 列名有空格，需要处理下
data.isnull().sum()

订单编号           0
总金额            0
买家实际支付金额       0
收货地址           0
订单创建时间         0
订单付款时间      3923
退款金额           0
dtype: int64

### 3.转换数据类型：确保所有列的数据类型都正确（例如，将时间列转换为 datetime 类型）。

In [4]:
# 将时间列转换为 datetime 类型
data['订单创建时间'] = pd.to_datetime(data['订单创建时间'])
data['订单付款时间'] = pd.to_datetime(data['订单付款时间'], errors='coerce')  # 如果有无法转换的值，可以使用 errors='coerce'

In [5]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaT,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaT,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   订单编号      28010 non-null  int64         
 1   总金额       28010 non-null  float64       
 2   买家实际支付金额  28010 non-null  float64       
 3   收货地址      28010 non-null  object        
 4   订单创建时间    28010 non-null  datetime64[ns]
 5   订单付款时间    24087 non-null  datetime64[ns]
 6   退款金额      28010 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(1)
memory usage: 1.5+ MB


### 4. 去除重复项：检查并删除重复的记录。

In [7]:
# 检查重复项
data.duplicated().sum()

0

In [8]:
# 删除重复项
data = data.drop_duplicates()
data.duplicated().sum()

0

# 第二步：数据分析及可视化

## 1. 描述性统计：使用 describe() 方法获取数值列的统计摘要。

In [9]:
# 获取数值列的统计摘要
data.describe()

,订单编号,总金额,买家实际支付金额,退款金额
count,28010.000000,28010.000000,28010.000000,28010.000000
mean,14005.500000,106.953253,67.921712,20.433271
std,8085.934856,1136.587094,151.493434,71.501963
min,1.000000,1.000000,0.000000,0.000000
25%,7003.250000,38.000000,0.000000,0.000000
50%,14005.500000,75.000000,45.000000,0.000000
75%,21007.750000,119.000000,101.000000,0.000000
max,28010.000000,188320.000000,16065.000000,3800.000000


In [10]:
# 整体情况
result = {}
result['总订单数'] = data['订单编号'].count()  
result['已完成订单数'] = data['订单编号'][data['订单付款时间'].notnull()].count()  
result['未付款订单数'] = data['订单编号'][data['订单付款时间'].isnull()].count()  
result['退款订单数'] = data['订单编号'][data['退款金额'] > 0].count()  
result['总订单金额'] = data['总金额'][data['订单付款时间'].notnull()].sum()  
result['总退款金额'] = data['退款金额'][data['订单付款时间'].notnull()].sum()  
result['总实际收入金额'] = data['买家实际支付金额'][data['订单付款时间'].notnull()].sum()  
result

{'总订单数': 28010,
 '已完成订单数': 24087,
 '未付款订单数': 3923,
 '退款订单数': 5646,
 '总订单金额': 2474823.0700000003,
 '总退款金额': 572335.9199999999,
 '总实际收入金额': 1902487.1500000001}

In [11]:
table = Table()
headers = ['总订单数', '总订单金额', '已完成订单数', '总实际收入金额', '退款订单数', '总退款金额', '成交率', '退货率']
rows = [
    [
        result['总订单数'], f"{result['总订单金额']/10000:.2f} 万", result['已完成订单数'], f"{result['总实际收入金额']/10000:.2f} 万",
        result['退款订单数'], f"{result['总退款金额']/10000:.2f} 万", 
        f"{result['已完成订单数']/result['总订单数']:.2%}",
        f"{result['退款订单数']/result['已完成订单数']:.2%}",
    ]
]
table.add(headers, rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='整体情况')
)

table.render_notebook()

总订单数,总订单金额,已完成订单数,总实际收入金额,退款订单数,总退款金额,成交率,退货率
28010,247.48 万,24087,190.25 万,5646,57.23 万,85.99%,23.44%


成交率 (85.99%)：成交率较高，表示订单的完成率较高，大部分订单都成功完成了付款。这对于卖家来说是一个积极的信号，表明顾客的购买意愿和付款成功率较高。
退款率 (23.44%)：退款率较高，表明有较多的订单在完成付款后发生了退款。这可能需要卖家关注和分析退款的原因，采取措施减少退款率，例如改进产品质量、优化售后服务等。
降低退款率：需要进一步分析退款的原因，找出主要问题所在，并采取相应措施减少退款。例如，改善产品质量、提高客户服务水平、优化退货政策等。

## 2. 分组分析：根据有意义的字段（例如收货地址）进行分组，计算各组的统计数据，并进行可视化

### 2.1 地区分析

In [12]:
# 创建映射表，将数据集中的省份名称映射到 Pyecharts 的标准名称
mapping = {
    '上海': '上海市',
    '北京': '北京市',
    '天津': '天津市',
    '重庆': '重庆市',
}

# 将数据集中的省份名称替换为标准名称
data['收货地址'] = data['收货地址'].replace(mapping)

In [13]:
# 按收货地址分组，计算每个收货地址的订单总金额和订单数量
grouped = data.groupby('收货地址').agg({'总金额': 'sum', '订单编号': 'count'}).rename(columns={'订单编号': '订单数量'})

# 创建包含额外省份的 DataFrame
extra_provinces = pd.DataFrame({
    '收货地址': ['澳门特别行政区', '香港特别行政区', '台湾省', '南海诸岛'],
    '总金额': [0, 0, 0, 0],
    '订单数量': [0, 0, 0, 0]
})
# 合并原始 grouped 数据和 extra_provinces 数据
grouped = pd.concat([grouped, extra_provinces.set_index('收货地址')])

grouped

,总金额,订单数量
收货地址,,
上海市,544907.63,3353
云南省,75769.32,778
内蒙古自治区,36827.00,215
北京市,231055.49,2054
吉林省,42040.92,401
四川省,188948.12,2019
天津市,124564.24,1153
宁夏回族自治区,4804.92,42
安徽省,61378.67,609


In [14]:
result21 = grouped['订单数量'].to_dict()

c = (
    Map()
    .add("订单量", [*result21.items()], "china", is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='不同收货地址的订单数量'),
        visualmap_opts=opts.VisualMapOpts(max_=grouped['订单数量'].max()),
    )
)
c.render_notebook()

总销售额和订单数量分布：
上海市的总销售额最高，达 544907.63 元，订单数量为 3353，表明上海市是最主要的市场之一。
其次是北京市，总销售额为 231055.49 元，订单数量为 2054。
广东省也表现突出，总销售额为 227855.28 元，订单数量为 2463。

对于上海市、北京市和广东省等主要市场，卖家应继续加大投入和关注，优化产品和服务，以保持高销售额和订单数量。
对于销售额较低的地区，如西藏自治区、青海省等，卖家可以分析原因，如物流、产品适配性等，尝试优化策略以提高销售额。

## 2.2 时间分析

In [15]:
result22 = data.groupby(data['订单创建时间'].apply(lambda x: x.strftime("%Y-%m-%d"))).agg({'订单编号':'count'}).to_dict()['订单编号']
c = (
    Line()
    .add_xaxis(list(result22.keys()))
    .add_yaxis("订单量", list(result22.values()))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="每日订单量走势"))
)
c.render_notebook()

从上图来看，2月份上半月由于受新冠疫情影响，订单量比较少，随着复工开展，下半月的订单量增长明显

In [16]:
result23 = data.groupby(data['订单创建时间'].apply(lambda x: x.strftime("%H"))).agg({'订单编号':'count'}).to_dict()['订单编号']
x = [*result23.keys()]
y = [*result23.values()]
c = (
    Bar()
    .add_xaxis(x)
    .add_yaxis("订单量", y)
    .set_global_opts(title_opts=opts.TitleOpts(title="每小时订单量走势"))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="峰值"),
                opts.MarkPointItem(name="第二峰值", coord=[x[15], y[15]], value=y[15]),
                opts.MarkPointItem(name="第三峰值", coord=[x[10], y[10]], value=y[10]),
            ]
        ),
    )
)
c.render_notebook()

从每小时订单量走势来看，一天中有3个高峰期（10点、15点、21点），其中21点-22点之间是一天中订单量最多的时候。对于卖家的指导意义就是，为了提高订单量，高峰期时应该尽量保证客服的回复速度，尤其是晚上21点-22点之间。

In [17]:
s = data['订单付款时间'] - data['订单创建时间']
s[s.notnull()].apply(lambda x: x.seconds / 60 ).mean()  # 从下单到付款的平均耗时为 7.7 分钟

7.7399046511949745

In [18]:
# 计算每个订单的下单到付款的时间（分钟）
data['下单到付款时间'] = (pd.to_datetime(data['订单付款时间']) - pd.to_datetime(data['订单创建时间'])).apply(lambda x: x.total_seconds() / 60 if pd.notnull(x) else np.nan)

# 计算每个省份的平均下单时间
province_avg_time = data.groupby('收货地址')['下单到付款时间'].mean().reset_index()
province_avg_time.columns = ['省份', '平均下单时间']

# 添加缺失的省份并将平均下单时间设为0
missing_provinces = ['澳门特别行政区', '香港特别行政区', '台湾省', '南海诸岛']
missing_data = pd.DataFrame({'省份': missing_provinces, '平均下单时间': [0]*len(missing_provinces)})

# 合并数据
province_avg_time = pd.concat([province_avg_time, missing_data], ignore_index=True)

# 打印表格
province_avg_time_styled = province_avg_time.style.hide(axis='index')
province_avg_time_styled

省份,平均下单时间
上海市,7.500103
云南省,11.142304
内蒙古自治区,14.261458
北京市,6.042768
吉林省,6.702133
四川省,6.178301
天津市,9.332913
宁夏回族自治区,0.636250
安徽省,8.453883
山东省,8.588522


In [19]:
# 将结果转换为字典
result_avg_time = province_avg_time.set_index('省份')['平均下单时间'].to_dict()

# 绘制地图
c = (
    Map()
    .add("平均下单时间", [*result_avg_time.items()], "china", is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='不同省份的平均下单时间（分钟）'),
        visualmap_opts=opts.VisualMapOpts(max_=province_avg_time['平均下单时间'].max())
    )
)

# 渲染图表
c.render_notebook()

平均下单时间差异显著：各省份的平均下单时间差异较大，最长的平均下单时间出现在广西壮族自治区（18.275921分钟），而最短的出现在西藏自治区（0.150分钟）。
对于平均下单时间较长的省份，如广西、新疆和内蒙古，卖家需要关注这些地区的物流和支付效率。可能需要优化物流渠道和支付流程，以缩短下单到付款的时间，提高客户满意度。
卖家可以根据不同省份的平均下单时间，制定有针对性的区域发展策略。例如，在平均下单时间较长的地区加大市场投入，改善服务质量，提升品牌形象，从而吸引更多的客户下单。